# data processing into a single dataframe


In [8]:
import pandas as pd

df_all = pd.read_csv('../data/chr14.out_scores.tsv', sep='\t', encoding = "UTF-8")
df_all = df_all.set_index(['chr','pos', 'ref', 'alt']) #index by identifier

for c in range (17,22):
    #print(c)
    path = '../data/chr' + str(c) + '.out_scores.tsv'
    df = pd.read_csv(path, sep='\t', encoding = "UTF-8")
    df = df.set_index(['chr','pos', 'ref', 'alt']) #index by identifier
    df_all= df_all.append(df)
    
print('concat shape', df_all.shape)

df = df_all
print("shape before", df.shape)
#print(df.head)
df = df.replace({'.': 0})  ## FIXME: atm replace with nan and drop: later find a way to impute, maybe 0 but not sure
#print("SUM", pd.isnull(df).sum())  ## Use this to see how many elements have nan 
df = df.dropna()
print("shape after dropping all data with score = '.'", df.shape)
#print(df.head())

## still contains some mixed labels, see them by: df[~df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]

## Map labels to 1/0 and remove Unkn
clinvar_labels={
    'Pathogenic': 1,
    'Likely_pathogenic': 1,
    'Pathogenic/Likely_pathogenic': 1,
    'Benign/Likely_benign': 0,
    'Likely_benign': 0,
    'Benign': 0
}

#df.head()
df = df[df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]  # best to drop everything that isn't in the dict
df['Problematic'] = df['clinvar_clnsig'].map(clinvar_labels)

df= df.drop(columns=['clinvar_clnsig'])#, 'clinvar_id'])  #tbh, I don't know why I kept them. consider adding to index

print("shape after dropping Unk", df.shape)
#df.head()

### pickle to reuse dataframe:
df.to_pickle("./pickled_df/all_scores_chr14,22.pkl")

concat shape (43207, 28)
shape before (43207, 28)
shape after dropping all data with score = '.' (43207, 28)
shape after dropping Unk (14929, 28)
